In [1]:
# Requires: pip install cryptography argon2-cffi
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from argon2 import PasswordHasher
import os
import json
import base64

# ----- Utilities -----
def b64(x: bytes) -> str:
    return base64.b64encode(x).decode('ascii')

def ub64(s: str) -> bytes:
    return base64.b64decode(s.encode('ascii'))

# ----- Step A: derive master key from password (Argon2) -----
ph = PasswordHasher(time_cost=2, memory_cost=102400, parallelism=8)  # tune for environment

# Use Argon2 only to verify/produce stored hash for authentication. To derive a binary master key,
# generate a random salt (store it) and use a KDF (HKDF) over a secret seed. Here we show a simple model.

def generate_master_key_from_password(password: str):
    # 1) Argon2: store verification hash for login (not used as master key directly)
    pwd_hash = ph.hash(password)  # store this in user DB for auth
    # 2) Create a separate random seed and encrypt/derive master_key with HKDF
    #    In a real system, you would derive a key material using a secure KDF (e.g., scrypt/argon + HKDF).
    seed = os.urandom(32)   # store encrypted/securely in KMS or wrapped with password-derived key
    # For illustration, we combine password and seed by HMAC/HKDF step:
    hkdf = HKDF(algorithm=hashes.SHA256(), length=32, salt=seed, info=b"ctc-master-key")
    master_key = hkdf.derive(password.encode('utf-8'))
    # store (pwd_hash, seed) in DB; master_key kept in memory or in KMS
    return { "pwd_hash": pwd_hash, "seed": b64(seed), "master_key": master_key }

# ----- Step B: per-message encryption using HKDF + AES-GCM -----
def encrypt_message(master_key: bytes, plaintext: bytes, aad: bytes = b"", nonce=None):
    # Use a unique nonce per message (12 bytes recommended for AES-GCM)
    if nonce is None:
        nonce = os.urandom(12)
    # Derive per-message key to limit damage of reuse/compromise
    hkdf = HKDF(algorithm=hashes.SHA256(), length=32, salt=nonce, info=b"ctc-per-message")
    per_key = hkdf.derive(master_key)
    aesgcm = AESGCM(per_key)
    ct = aesgcm.encrypt(nonce, plaintext, aad)  # returns ciphertext||tag
    # return components (store nonce and aad metadata)
    return { "nonce": b64(nonce), "aad": b64(aad), "ciphertext": b64(ct) }

def decrypt_message(master_key: bytes, envelope: dict):
    nonce = ub64(envelope["nonce"])
    aad = ub64(envelope["aad"])
    ct = ub64(envelope["ciphertext"])
    hkdf = HKDF(algorithm=hashes.SHA256(), length=32, salt=nonce, info=b"ctc-per-message")
    per_key = hkdf.derive(master_key)
    aesgcm = AESGCM(per_key)
    plain = aesgcm.decrypt(nonce, ct, aad)
    return plain

# ----- Demo usage -----
if __name__ == "__main__":
    password = "correct horse battery staple"  # encourage long passphrases!
    obj = generate_master_key_from_password(password)
    mk = obj["master_key"]

    message = b"We meet at dawn. WEATHER REPORT."
    aad = b"sender:alice|msgid:001"

    envelope = encrypt_message(mk, message, aad)
    print("ENVELOPE JSON\n", json.dumps(envelope, indent=2))

    recovered = decrypt_message(mk, envelope)
    print("Recovered:", recovered.decode())

ENVELOPE JSON
 {
  "nonce": "5SBAB6tx8RwJCN/z",
  "aad": "c2VuZGVyOmFsaWNlfG1zZ2lkOjAwMQ==",
  "ciphertext": "eIu2P9NzYrA3/mrkbV5BsFpKpSkQg8x15jeakEX3ksF3pD3VewIzzcaUaK2m+/4N"
}
Recovered: We meet at dawn. WEATHER REPORT.


In [2]:
import os
import hashlib
from base64 import urlsafe_b64encode, urlsafe_b64decode

used_nonces = set()  # prevent reuse

def generate_key():
    """Generate a random 256-bit key"""
    return os.urandom(32)

def encrypt_message(message, master_key):
    """Encrypt with a random nonce and SHA256 as pseudo-cipher"""
    nonce = os.urandom(8)  # short unique ID
    if nonce in used_nonces:
        raise ValueError("Nonce reuse detected! Encryption aborted.")
    used_nonces.add(nonce)

    # Combine key, nonce, and message for pseudo-encryption
    combined = master_key + nonce + message.encode()
    cipher = hashlib.sha256(combined).digest()

    # Output includes the nonce (like real ciphers)
    return urlsafe_b64encode(nonce + cipher).decode()

def decrypt_message(ciphertext, master_key):
    """Reverse process (not truly decryptable—demo only)"""
    data = urlsafe_b64decode(ciphertext.encode())
    nonce, cipher = data[:8], data[8:]
    print(f"Decrypting with nonce: {nonce.hex()}")
    print("This demo simulates how nonce must match exactly for real ciphers!")

# ---- Demo ----
key = generate_key()
msg1 = "Meet me at midnight."
msg2 = "Same message, different nonce."

cipher1 = encrypt_message(msg1, key)
cipher2 = encrypt_message(msg1, key)
cipher3 = encrypt_message(msg2, key)

print("Ciphertext 1:", cipher1)
print("Ciphertext 2:", cipher2)
print("Ciphertext 3:", cipher3)
print("\nNotice: even identical messages produce completely different ciphertexts!\n")

decrypt_message(cipher1, key)

Ciphertext 1: QZnMjmgYK-xtadErCzZNIAyUAq6tMbMK8GC4NKUspxX53OnDJqjNbw==
Ciphertext 2: UbRKIQ7STOOx-SSi8itnM2SS8nfxQ9GtEF7Q4QVjOrFljClKZsK9Zg==
Ciphertext 3: W0peKkpYqWhjr56t2ZN7-zD6Pa9XgkQDGyFxCIn3yNQhjyH8FTR3eQ==

Notice: even identical messages produce completely different ciphertexts!

Decrypting with nonce: 4199cc8e68182bec
This demo simulates how nonce must match exactly for real ciphers!
